In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pprint import pprint

from terminusdb_client import WOQLQuery as WQ

from ads_query_eval.config import get_terminus_client, get_terminus_config
from ads_query_eval.app.bootstrap import QUERIES

client = get_terminus_client()

# scratch

# util

In [ ]:
for name, defn in client.get_existing_classes().items():
    print(name)
    #pprint(defn)
    #print()

In [ ]:
import requests

def tget(url):
    client._check_connection()
    return requests.get(
        url,
        headers=client._default_headers,
        auth=client._auth(),
    )

def tget_ttl(dbid, which=None):
    assert which in {"schema","instance"}
    rv = tget(f"http://terminus:6363/api/triples/admin/{dbid}/local/branch/main/{which}?format=turtle")
    for line in rv.json().split('\n'):
        print(line)
    return rv

tget_ttl(client.db, which="schema")
#tget_ttl(client.db, which="instance")